In [2]:
pip install msvc-runtime

   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.9 MB 682.7 kB/s eta 0:00:03
    --------------------------------------- 0.0/1.9 MB 495.5 kB/s eta 0:00:04
   - -------------------------------------- 0.1/1.9 MB 375.8 kB/s eta 0:00:05
   - -------------------------------------- 0.1/1.9 MB 375.8 kB/s eta 0:00:05
   - -------------------------------------- 0.1/1.9 MB 328.6 kB/s eta 0:00:06
   -- ------------------------------------- 0.1/1.9 MB 368.6 kB/s eta 0:00:05
   -- ------------------------------------- 0.1/1.9 MB 368.6 kB/s eta 0:00:05
   -- ------------------------------------- 0.1/1.9 MB 357.2 kB/s eta 0:00:06
   --- ------------------------------------ 0.2/1.9 MB 399.3 kB/s eta 0:00:05
   --- ------------------------------------ 0.2/1.9 MB 399.3 kB/s eta 0:00:05
   --- ------------------------------------ 0.2/1.9 MB 359.7 kB/s eta 0:00:05
   -

In [3]:
pip install mediapipe --user

  Using cached mediapipe-0.10.20-cp311-cp311-win_amd64.whl.metadata (9.9 kB)
  Using cached jax-0.4.37-py3-none-any.whl.metadata (22 kB)
  Using cached jaxlib-0.4.36-cp311-cp311-win_amd64.whl.metadata (1.1 kB)
  Using cached opencv_contrib_python-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached sounddevice-0.5.1-py3-none-win_amd64.whl.metadata (1.4 kB)
Using cached mediapipe-0.10.20-cp311-cp311-win_amd64.whl (51.0 MB)
Using cached sounddevice-0.5.1-py3-none-win_amd64.whl (363 kB)
Using cached jax-0.4.37-py3-none-any.whl (2.2 MB)
Using cached jaxlib-0.4.36-cp311-cp311-win_amd64.whl (63.3 MB)
Using cached opencv_contrib_python-4.10.0.84-cp37-abi3-win_amd64.whl (45.5 MB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import cv2
import mediapipe as mp
from math import hypot
import screen_brightness_control as sbc
import numpy as np


In [2]:
#initializing models
mpHands=mp.solutions.hands
hands=mpHands.Hands(static_image_mode=False,
    model_complexity=1,
    min_detection_confidence=0.75,
    min_tracking_confidence=0.75,
    max_num_hands=2)
Draw=mp.solutions.drawing_utils

In [ ]:
#Start capturing video from webcam
cap = cv2.VideoCapture(0)

while True:
    #Read video frame by frame
    _,frame = cap.read()

    #Flip image
    frame=cv2.flip(frame,1)

    #convert BGR image to RGB image
    frameRGB = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)

    #Process the RGB image
    Process = hands.process(frameRGB)

    landmarkList = []
    # if hands are present in image(frame)
    if Process.multi_hand_landmarks:
        # detect handmarks
        for handlm in Process.multi_hand_landmarks:
            for _id,landmarks in enumerate(handlm.landmark):
                #store height and width of image
                height,width,color_channels = frame.shape

                #calculate and append x,y coordinatesQQQ
                # of handmarks from image(frame) to lmList
                x,y = int(landmarks.x*width),int(landmarks.y*height)
                landmarkList.append([_id,x,y])
                
            #draw landmarks
            Draw.draw_landmarks(frame,handlm,mpHands.HAND_CONNECTIONS)
            
    #If landmarks list is not empty
    if landmarkList !=[]:
        #store x,y coordinates of (tip of) thumb
        x_1,y_1 = landmarkList[4][1],landmarkList[4][2]

        #store x,y coordinates of (tip of) index finger
        x_2,y_2 = landmarkList[8][1],landmarkList[8][2]

        #draw circle on thumb and index finger tip
        cv2.circle(frame,(x_1,y_1),7,(0,255,0),cv2.FILLED)
        cv2.circle(frame,(x_2,y_2),7,(0,255,0),cv2.FILLED)

        #Draw line from tip of thumb to tip of index finger
        cv2.line(frame,(x_1,y_1),(x_2,y_2),(0,255,0),3)

        #calculate square root of the sum
        #of squares of the specified arguments
        L=hypot(x_2-x_1,y_2-y_1)

        # 1-D linear interpolant to a function
        # with given discrete data points
        # (Hand range 15-220, Brightness range 0-100),
        # evaluated at length
        b_level = np.interp(L,[15,220],[0,100])

        # set brightness
        sbc.set_brightness(int(b_level))
        
    #Display video and when 'q' is entered,
    #destroy the window
    cv2.imshow('Image',frame)
    if cv2.waitKey(1) & 0xff == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()